In [1]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

bytedance


In [2]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

feature_label_First1ELast1E_test.txt  query_titleConcat_label.txt
lost+found			      query_titleConcat.txt
pre_csv.txt


In [3]:
# 查看当前kernerl下的package
# !pip list --format=columns

Package                       Version    
----------------------------- -----------
absl-py                       0.7.1      
alabaster                     0.7.12     
allennlp                      0.8.0      
altair                        1.2.0      
arrow                         0.12.1     
asn1crypto                    0.24.0     
astor                         0.8.0      
atomicwrites                  1.3.0      
attrs                         19.1.0     
audioread                     2.1.8      
aws-xray-sdk                  0.95       
awscli                        1.16.193   
Babel                         2.7.0      
backcall                      0.1.0      
beautifulsoup4                4.7.1      
blaze                         0.10.1     
bleach                        3.1.0      
blinker                       1.4        
bokeh                         1.0.2      
boto                          2.49.0     
boto3                         1.9.183    
botocore                      1.12

In [4]:
# 显示cell运行时长
%load_ext klab-autotime

In [5]:
import numpy as np
import pandas as pd
import Levenshtein
import math
import time
from gensim.models import Word2Vec, Doc2Vec
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import StratifiedKFold, train_test_split
import lightgbm as lgb
import multiprocessing
import gc

time: 941 ms


In [7]:
path="/home/kesci/input/bytedance/"
train = pd.read_csv(path+'train_final.csv', header=None, chunksize=100000000, names=['query_id', 'query', 'title_id', 'title', 'label'])
test = pd.read_csv(path+'test_final_part1.csv', header=None, names=['query_id', 'query', 'title_id', 'title'])
pre_csv = test[['query_id', 'title_id']]
pre_csv.to_csv('pre_csv.txt', index=False)
del pre_csv
gc.collect()

7

time: 1min 38s


In [8]:
#去除'\t'符号
test['title'] = test['title'].apply(lambda x: x[:-1]if x[-1] == '\t' else x)

time: 9.45 s


In [9]:
def lcsubstr_lens(s1, s2): 
    m=[[0 for i in range(len(s2)+1)]  for j in range(len(s1)+1)]  #生成0矩阵，为方便后续计算，比字符串长度多了一列
    mmax=0   #最长匹配的长度
    p=0  #最长匹配对应在s1中的最后一位
    for i in range(len(s1)):
        for j in range(len(s2)):
            if s1[i]==s2[j]:
                m[i+1][j+1]=m[i][j]+1
                if m[i+1][j+1]>mmax:
                    mmax=m[i+1][j+1]
                    p=i+1
    return mmax


#
def lcseque_lens(s1, s2): 
     # 生成字符串长度加1的0矩阵，m用来保存对应位置匹配的结果
    m = [ [ 0 for x in range(len(s2)+1) ] for y in range(len(s1)+1) ] 
    # d用来记录转移方向
    d = [ [ None for x in range(len(s2)+1) ] for y in range(len(s1)+1) ] 

    for p1 in range(len(s1)): 
        for p2 in range(len(s2)): 
            if s1[p1] == s2[p2]:            #字符匹配成功，则该位置的值为左上方的值加1
                m[p1+1][p2+1] = m[p1][p2]+1
                d[p1+1][p2+1] = 'ok'          
            elif m[p1+1][p2] > m[p1][p2+1]:  #左值大于上值，则该位置的值为左值，并标记回溯时的方向
                m[p1+1][p2+1] = m[p1+1][p2] 
                d[p1+1][p2+1] = 'left'          
            else:                           #上值大于左值，则该位置的值为上值，并标记方向up
                m[p1+1][p2+1] = m[p1][p2+1]   
                d[p1+1][p2+1] = 'up'         
    (p1, p2) = (len(s1), len(s2)) 
    s = [] 
    while m[p1][p2]:    #不为None时
        c = d[p1][p2]
        if c == 'ok':   #匹配成功，插入该字符，并向左上角找下一个
            s.append(s1[p1-1])
            p1 -= 1
            p2 -= 1 
        if c == 'left':  #根据标记，向左找下一个
            p2 -= 1
        if c == 'up':   #根据标记，向上找下一个
            p1 -= 1
    return len(s)


def levenshtein(first, second):
        ''' 编辑距离算法（LevD） 
            Args: 两个字符串
            returns: 两个字符串的编辑距离 int
        '''
        if len(first) > len(second):
            first, second = second, first
        if len(first) == 0:
            return len(second)
        if len(second) == 0:
            return len(first)
        first_length = len(first) + 1
        second_length = len(second) + 1
        distance_matrix = [list(range(second_length)) for x in range(first_length)]
        # print distance_matrix
        for i in range(1, first_length):
            for j in range(1, second_length):
                deletion = distance_matrix[i - 1][j] + 1
                insertion = distance_matrix[i][j - 1] + 1
                substitution = distance_matrix[i - 1][j - 1]
                if first[i - 1] != second[j - 1]:
                    substitution += 1
                distance_matrix[i][j] = min(insertion, deletion, substitution)
                # print distance_matrix
        return distance_matrix[first_length - 1][second_length - 1]
    
    
#ratio
def ratio(first, second):
        ''' 编辑距离算法（LevD） 
            Args: 两个字符串
            returns: 两个字符串的编辑距离 int
        '''
        if len(first) > len(second):
            first, second = second, first
        if len(first) == 0:
            return len(second)
        if len(second) == 0:
            return len(first)
        first_length = len(first) + 1
        second_length = len(second) + 1
        sum_len = first_length + second_length 
        distance_matrix = [list(range(second_length)) for x in range(first_length)]
        # print distance_matrix
        for i in range(1, first_length):
            for j in range(1, second_length):
                deletion = distance_matrix[i - 1][j] + 1
                insertion = distance_matrix[i][j - 1] + 1
                substitution = distance_matrix[i - 1][j - 1]
                if first[i - 1] != second[j - 1]:
                    substitution += 2
                distance_matrix[i][j] = min(insertion, deletion, substitution)
                # print distance_matrix
        return np.float32((sum_len - distance_matrix[first_length - 1][second_length - 1])/sum_len)
   
#余弦相似度
def compute_cosine(text_a, text_b):
    # 找单词及词频
    words1 = text_a.split(' ')
    words2 = text_b.split(' ')
    # print(words1)
    words1_dict = {}
    words2_dict = {}
    for word in words1:
        # word = word.strip(",.?!;")
        word = word.lower()
        # print(word)
        if word != '' and word in words1_dict: # 这里改动了
            num = words1_dict[word]
            words1_dict[word] = num + 1
        elif word != '':
            words1_dict[word] = 1
        else:
            continue
    for word in words2:
        # word = word.strip(",.?!;")
        word = word.lower()
        if word != '' and word in words2_dict:
            num = words2_dict[word]
            words2_dict[word] = num + 1
        elif word != '':
            words2_dict[word] = 1
        else:
            continue

    # 排序
    dic1 = sorted(words1_dict.items(), key=lambda asd: asd[1], reverse=True)
    dic2 = sorted(words2_dict.items(), key=lambda asd: asd[1], reverse=True)

    # 得到词向量
    words_key = []
    for i in range(len(dic1)):
        words_key.append(dic1[i][0])  # 向数组中添加元素
    for i in range(len(dic2)):
        if dic2[i][0] in words_key:
            # print 'has_key', dic2[i][0]
            pass
        else:  # 合并
            words_key.append(dic2[i][0])
    # print(words_key)
    vect1 = []
    vect2 = []
    for word in words_key:
        if word in words1_dict:
            vect1.append(words1_dict[word])
        else:
            vect1.append(0)
        if word in words2_dict:
            vect2.append(words2_dict[word])
        else:
            vect2.append(0)

    # 计算余弦相似度
    sum = 0
    sq1 = 0
    sq2 = 0
    for i in range(len(vect1)):
        sum += vect1[i] * vect2[i]
        sq1 += pow(vect1[i], 2)
        sq2 += pow(vect2[i], 2)
    try:
        result = round(float(sum) / (math.sqrt(sq1) * math.sqrt(sq2)), 2)
    except ZeroDivisionError:
        result = 0.0
    return result



#皮尔逊系数
def Pehrson(text_a, text_b):
    # 找单词及词频
    words1 = text_a.split(' ')
    words2 = text_b.split(' ')
    # print(words1)
    words1_dict = {}
    words2_dict = {}
    for word in words1:
        # word = word.strip(",.?!;")
        word = word.lower()
        # print(word)
        if word != '' and word in words1_dict: # 这里改动了
            num = words1_dict[word]
            words1_dict[word] = num + 1
        elif word != '':
            words1_dict[word] = 1
        else:
            continue
    for word in words2:
        # word = word.strip(",.?!;")
        word = word.lower()
        if word != '' and word in words2_dict:
            num = words2_dict[word]
            words2_dict[word] = num + 1
        elif word != '':
            words2_dict[word] = 1
        else:
            continue

    # 排序
    dic1 = sorted(words1_dict.items(), key=lambda asd: asd[1], reverse=True)
    dic2 = sorted(words2_dict.items(), key=lambda asd: asd[1], reverse=True)

    # 得到词向量
    words_key = []
    for i in range(len(dic1)):
        words_key.append(dic1[i][0])  # 向数组中添加元素
    for i in range(len(dic2)):
        if dic2[i][0] in words_key:
            # print 'has_key', dic2[i][0]
            pass
        else:  # 合并
            words_key.append(dic2[i][0])
    # print(words_key)
    vect1 = []
    vect2 = []
    for word in words_key:
        if word in words1_dict:
            vect1.append(words1_dict[word])
        else:
            vect1.append(0)
        if word in words2_dict:
            vect2.append(words2_dict[word])
        else:
            vect2.append(0)

    # 计算Pehrson
    x = np.vstack([vect1, vect2])
    return np.corrcoef(x)[0][1]



#
def getBayesSmoothParam(origion_rate):
    origion_rate_mean = origion_rate.mean()
    origion_rate_var = origion_rate.var()
    alpha = origion_rate_mean / origion_rate_var * (origion_rate_mean * (1 - origion_rate_mean) - origion_rate_var)
    beta = (1 - origion_rate_mean) / origion_rate_var * (origion_rate_mean * (1 - origion_rate_mean) - origion_rate_var)
    return alpha, beta


# 统计单维度的转化率特征
def get_single_dimension_rate_train_feature(train_df, fea_set):
    skf = StratifiedKFold(n_splits=5, random_state=2019, shuffle=True)
    for fea in fea_set:
        train_temp_df = pd.DataFrame()
        for index, (train_index, test_index) in enumerate(skf.split(train_df, train_df['label'])):
            temp_df = train_df[[fea, 'label']].iloc[train_index].copy()
            temp_pivot_table = pd.pivot_table(temp_df, index=fea, values='label', aggfunc={len, np.mean, np.sum})
            temp_pivot_table.reset_index(inplace=True)
            temp_pivot_table.rename(columns={'len':fea + '_count', 'mean':fea + '_rate', 'sum':fea + '_click_number'}, inplace=True)
            alpha, beta = getBayesSmoothParam(temp_pivot_table[fea + '_rate'])
            temp_pivot_table[fea + '_rate'] = (temp_pivot_table[fea + '_click_number'] + alpha) / (temp_pivot_table[fea + '_count'] + alpha + beta)
            fea_df = train_df.iloc[test_index].copy()
            fea_df = pd.merge(fea_df, temp_pivot_table, on=fea, how='left')
            train_temp_df = pd.concat([train_temp_df, fea_df])
        print(fea + ' : finish!!!')
        train_df = train_temp_df
        train_df.sort_index(by='index', ascending=True, inplace=True)
    return train_df



# 统计双维度交叉转化率
def get_jiaocha_dimension_rate_train_feature(train_df, fea_set):
    skf = StratifiedKFold(n_splits=5, random_state=2019, shuffle=True)
    for i in range(len(fea_set)):
        for j in range((i+1), len(fea_set)):
            fea1 = fea_set[i]
            fea2 = fea_set[j]
            train_temp_df = pd.DataFrame()
            for index, (train_index, test_index) in enumerate(skf.split(train_df, train_df['label'])):
                temp_df = train_df[[fea1, fea2, 'label']].iloc[train_index].copy()
                temp_pivot_table = pd.pivot_table(temp_df, index=[fea1, fea2], values='label', aggfunc={len, np.mean, np.sum})
                temp_pivot_table.reset_index(inplace=True)
                temp_pivot_table.rename(columns={'len':fea1 + '_' + fea2 + '_count', 'mean':fea1 + '_' + fea2 + '_rate', 'sum':fea1 + '_' + fea2 + '_click_number'}, inplace=True)
                alpha, beta = getBayesSmoothParam(temp_pivot_table[fea1 + '_' + fea2 + '_rate'])
                temp_pivot_table[fea1 + '_' + fea2 + '_rate'] = (temp_pivot_table[fea1 + '_' + fea2 + '_click_number'] + alpha) / (temp_pivot_table[fea1 + '_' + fea2 + '_count'] + alpha + beta)
                fea_df = train_df.iloc[test_index].copy()
                fea_df = pd.merge(fea_df, temp_pivot_table, on=[fea1, fea2], how='left')
                train_temp_df = pd.concat([train_temp_df, fea_df])
            print(fea1 + '_' + fea2 + ' : finish!!!')
            train_df = train_temp_df
            train_df.sort_index(by='index', ascending=True, inplace=True)
    return train_df


# 统计单维度的转化率特征
def get_single_dimension_rate_test_feature(train_df, valid_df, fea_set):
    for fea in fea_set:
        temp_df = train_df[[fea, 'label']].copy()
        temp_pivot_table = pd.pivot_table(temp_df, index=fea, values='label', aggfunc={len, np.mean, np.sum})
        temp_pivot_table.reset_index(inplace=True)
        temp_pivot_table.rename(columns={'len':fea + '_count', 'mean':fea + '_rate', 'sum':fea + '_click_number'}, inplace=True)
        alpha, beta = getBayesSmoothParam(temp_pivot_table[fea + '_rate'])
        temp_pivot_table[fea + '_rate'] = (temp_pivot_table[fea + '_click_number'] + alpha) / (temp_pivot_table[fea + '_count'] + alpha + beta)
        valid_df = pd.merge(valid_df, temp_pivot_table, on=fea, how='left')
        print(fea + ' : finish!!!')
    return valid_df

# 统计双维度交叉转化率
def get_jiaocha_dimension_rate_test_feature(train_df, valid_df, fea_set):
    for i in range(len(fea_set)):
        for j in range((i+1), len(fea_set)):
            fea1 = fea_set[i]
            fea2 = fea_set[j]
            temp_df = train_df[[fea1, fea2, 'label']].copy()
            temp_pivot_table = pd.pivot_table(temp_df, index=[fea1, fea2], values='label', aggfunc={len, np.mean, np.sum})
            temp_pivot_table.reset_index(inplace=True)
            temp_pivot_table.rename(columns={'len':fea1 + '_' + fea2 + '_count', 'mean':fea1 + '_' + fea2 + '_rate', 'sum':fea1 + '_' + fea2 + '_click_number'}, inplace=True)
            alpha, beta = getBayesSmoothParam(temp_pivot_table[fea1 + '_' + fea2 + '_rate'])
            temp_pivot_table[fea1 + '_' + fea2 + '_rate'] = (temp_pivot_table[fea1 + '_' + fea2 + '_click_number'] + alpha) / (temp_pivot_table[fea1 + '_' + fea2 + '_count'] + alpha + beta)
            print(fea1 + '_' + fea2 + ' : finish!!!')
            valid_df = pd.merge(valid_df, temp_pivot_table, on=[fea1, fea2], how='left')
    return valid_df




def getfeature(df):
    df.drop_duplicates(inplace=True)
    
    ite = 1
    since = time.time()
    
    #query 长度
    df['query_len'] = df.apply(lambda x: len(x[1].split(' ')), axis=1)
    df['query_len'] = df['query_len'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #query_unique 长度
    def get_query_unique(x):
        a = len(set(x[1].split(' '))-set(x[3].split(' ')))
        return a
    
    df['query_unique'] = df.apply(get_query_unique, axis=1)
    df['query_unique'] = df['query_unique'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #title 长度
    df['title_len'] = df.apply(lambda x: len(x[3].split(' ')), axis=1)
    df['title_len'] = df['title_len'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #title_unique 长度
    def get_query_unique(x):
        a = len(set(x[3].split(' '))-set(x[1].split(' ')))
        return a
    df['title_unique'] = df.apply(get_query_unique, axis=1)
    df['title_unique'] = df['title_unique'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #长度差、unique长度差
    df['dif'] = (df['query_len'] - df['title_len']).abs()
    df['dif'] = df['dif'].astype(np.int32)
    df['dif_unique'] = (df['query_unique'] - df['title_unique']).abs()
    df['dif_unique'] = df['dif_unique'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #query_title_same_len query和title相同的长度
    def query_title_same_len(x):
        a = len(x[3].split(' ') and x[1].split(' '))
        return a
    df['query_title_same_len'] = df.apply(query_title_same_len, axis=1)
    df['query_title_same_len'] = df['query_title_same_len'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1

    #query和title相同的长度在query的比率
    def samelen_query_rate(x):
        a = len(set(x[1].split(' ')) & set(x[3].split(' ')))
        return a/len(x[1].split(' '))
    df['samelen_query_rate'] = df.apply(samelen_query_rate, axis=1)
    df['samelen_query_rate'] = df['samelen_query_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #query和title相同的长度在title的比率
    def samelen_title_rate(x):
        a = len(set(x[1].split(' ')) & set(x[3].split(' ')))
        return a/len(x[3].split(' '))
    df['samelen_title_rate'] = df.apply(samelen_title_rate, axis=1)
    df['samelen_title_rate'] = df['samelen_title_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    
    #query和title总长度
    def q_t_all_len(x):
        a = len(x[1].split(' ')) + len(x[3].split(' '))
        return a
    df['q_t_all_len'] = df.apply(q_t_all_len, axis=1)
    df['q_t_all_len'] = df['q_t_all_len'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #query和title的unique长度
    def q_t_unique_len(x):
        a = len(set(x[1].split(' ') + x[3].split(' ')))
        return a
    df['q_t_unique_len'] = df.apply(q_t_unique_len, axis=1)
    df['q_t_unique_len'] = df['q_t_unique_len'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1

    
    #query和title总词数
    def q_t_all_word_len(x):
        a = len(set(x[1].split(' ')) | set(x[3].split(' ')))
        return a
    df['q_t_all_word_len'] = df.apply(q_t_all_word_len, axis=1)
    df['q_t_all_word_len'] = df['q_t_all_word_len'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #query和title不同词数
    df['q_t_diff_len'] = df.apply(lambda x: len(set(x[1].split(' '))^set(x[3].split(' '))), axis=1)
    df['q_t_diff_len'] = df['q_t_diff_len'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #query和title不同词数在query的比率
    def q_t_diff_q_rate(x):
        a = len(set(x[1].split(' '))^set(x[3].split(' ')))
        return np.float32(a/len(x[1].split(' ')))
    df['q_t_diff_q_rate'] = df.apply(q_t_diff_q_rate, axis=1)
    df['q_t_diff_q_rate'] = df['q_t_diff_q_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #query和title不同词数在title的比率
    def q_t_diff_t_rate(x):
        a = len(set(x[1].split(' '))^set(x[3].split(' ')))
        return np.float32(a/len(x[3].split(' ')))
    df['q_t_diff_t_rate'] = df.apply(q_t_diff_t_rate, axis=1)
    df['q_t_diff_t_rate'] = df['q_t_diff_t_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1

    
    #query在title中的不同词数
    def query_diff_title(x):
        a = len(set(x[1].split(' ')) - (set(x[1].split(' ')) & set(x[3].split(' '))))
        return np.int32(a)
    df['query_diff_title'] = df.apply(query_diff_title, axis=1)
    df['query_diff_title'] = df['query_diff_title'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #query在title中的不同词数在query的比率
    def query_diff_title_rate(x):
        a = len(set(x[1].split(' ')) - (set(x[1].split(' ')) & set(x[3].split(' '))))
        return np.float32(a/len(x[1].split(' ')))
    df['query_diff_title_rate'] = df.apply(query_diff_title_rate, axis=1)
    df['query_diff_title_rate'] = df['query_diff_title_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #query在title中的不同词数在title的比率
    def query_diff_title_rate(x):
        a = len(set(x[1].split(' ')) - (set(x[1].split(' ')) & set(x[3].split(' '))))
        return np.float32(a/len(x[3].split(' ')))
    df['query_diff_title_rate'] = df.apply(query_diff_title_rate, axis=1)
    df['query_diff_title_rate'] = df['query_diff_title_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #title在qurey中的不同词数
    def title_diff_query(x):
        a = len(set(x[3].split(' ')) - (set(x[1].split(' ')) & set(x[3].split(' '))))
        return np.int32(a)
    df['title_diff_query'] = df.apply(title_diff_query, axis=1)
    df['title_diff_query'] = df['title_diff_query'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #title在qurey中的不同词数在query的比率
    def title_diff_query_rate(x):
        a = len(set(x[3].split(' ')) - (set(x[1].split(' ')) & set(x[3].split(' '))))
        return np.float32(a/len(x[1].split(' ')))
    df['title_diff_query_rate'] = df.apply(title_diff_query_rate, axis=1)
    df['title_diff_query_rate'] = df['title_diff_query_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #title在qurey中的不同词数在title的比率
    def title_diff_query_rate(x):
        a = len(set(x[3].split(' ')) - (set(x[1].split(' ')) & set(x[3].split(' '))))
        return np.float32(a/len(x[3].split(' ')))
    df['title_diff_query_rate'] = df.apply(title_diff_query_rate, axis=1)
    df['title_diff_query_rate'] = df['title_diff_query_rate'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #Dice系数
    def dice(x):
        a1 = len(set(x[1].split(' ')) & set(x[3].split(' ')))
        a2 = len(set(x[1].split(' '))) + len(set(x[3].split(' ')))
        return np.float32(2*a1/a2)
    df['dice'] = df.apply(dice, axis=1)
    df['dice'] = df['dice'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #jaccord系数
    def jaccord(x):
        a1 = len(set(x[1].split(' ')) & set(x[3].split(' ')))
        a2 = len(set(x[1].split(' ')) | set(x[3].split(' ')))
        return np.float32(a1/a2)
    df['jaccord'] = df.apply(jaccord, axis=1)
    df['jaccord'] = df['jaccord'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #同一个query出现次数
    df['query_count'] = df.groupby(1)[1].transform('count')
    df['query_count'] = df['query_count'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #同一个title出现的次数
    df['title_count'] = df.groupby(3)[3].transform('count')
    df['title_count'] = df['title_count'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    
#    #query和labe的交叉点击
#    df['query_label_click'] = df.groupby([1, 4])[1].transform('count')
#    df['query_label_click'] = df['query_label_click'].astype(np.int32)
#    
#    #title和labe的交叉点击
#    df['title_label_click'] = df.groupby([3, 4])[3].transform('count')
#    df['title_label_click'] = df['title_label_click'].astype(np.int32)
    
    
    #lcseque_lens
    df['lcseque_lens'] = df.apply(lambda x: lcseque_lens(x[1], x[3]), axis=1)
    df['lcseque_lens'] = df['lcseque_lens'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #lcsubstr_lens
    df['lcsubstr_lens'] = df.apply(lambda x: lcsubstr_lens(x[1], x[3]), axis=1)
    df['lcsubstr_lens'] = df['lcsubstr_lens'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1

    #ratio
    df['ratio'] = df.apply(lambda x: Levenshtein.ratio(x[1], x[3]), axis=1)
    df['ratio'] = df['ratio'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    

    #Jaro距离
    df['jaro'] = df.apply(lambda x: Levenshtein.jaro(x[1], x[3]), axis=1)
    df['jaro'] = df['jaro'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    #jaro_winkler
    df['jaro_winkler'] = df.apply(lambda x: Levenshtein.jaro_winkler(x[1], x[3]), axis=1)
    df['jaro_winkler'] = df['jaro_winkler'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1


    #余弦相似度
    #使用tfidf向量求相似度
    ct = TfidfVectorizer()
    def cosine_similarity(x):
        ct.fit([x[1] + ' ' + x[3]])
        vect1 = ct.transform([x[1]]).toarray()[0]
        vect2 = ct.transform([x[3]]).toarray()[0]
        sum = 0
        sq1 = 0
        sq2 = 0
        for i in range(len(vect1)):
            sum += vect1[i] * vect2[i]
            sq1 += pow(vect1[i], 2)
            sq2 += pow(vect2[i], 2)
        try:
            result = round(float(sum) / (math.sqrt(sq1) * math.sqrt(sq2)), 2)
        except ZeroDivisionError:
            result = 0.0
        return result 
    df['cosine_similarity_tfidf'] = df.apply(lambda x: compute_cosine(x[1], x[3]), axis=1)
    df['cosine_similarity_tfidf'] = df['cosine_similarity_tfidf'].astype(np.float32)
    #使用词频向量求相似度
    df['cosine_similarity'] = df.apply(lambda x: compute_cosine(x[1], x[3]), axis=1)
    df['cosine_similarity'] = df['cosine_similarity'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    
    #编辑距离
    df['levenshtein'] = df.apply(lambda x: Levenshtein.distance(x[1], x[3]), axis=1)
    df['levenshtein'] = df['levenshtein'].astype(np.int32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    
    #皮尔逊相关系数Pehrson
    df['Pehrson'] = df.apply(lambda x:Pehrson(x[1], x[3]), axis=1)
    df['Pehrson'] = df['Pehrson'].fillna(df['Pehrson'].mean())
    df['Pehrson'] = df['Pehrson'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('{}complete in {:.0f}m {:.0f}s'.format(ite, time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    
    #计算两个字符串list的相似度
    df['list_dis'] = df.apply(lambda x: Levenshtein.seqratio(x[1].split(' '), x[3].split(' ')), axis=1)
    df['list_dis'] = df['list_dis'].astype(np.float32)
    
    time_elapsed = time.time() - since#######################################
    print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
    ite += 1
    
    
#     #Word2vec计算两个list的相似度
#     model_word = Word2Vec.load('./word2vec_groupby.model')
#     def word2vec_similar(x):
#         a = x[1].split(' ')
#         a = [i for i in a if i in model_word]
#         b = x[3].split(' ')
#         b = [i for i in b if i in model_word]  
#         if len(a) == 0 or len(b) == 0:
#               return 0
#         return model_word.n_similarity(a, b)
#     df['word_list_similarity'] = df.apply(lambda x: word2vec_similar(x), axis=1)
    
#     time_elapsed = time.time() - since#######################################
#     print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
#     ite += 1
    
    
#     #使用word2vec求句向量的均值和最大值
#     def getvec(x):
#         a = 0
#         n = 0
#         for i in x:
#             try:
#                 a += model_word[i]
#                 n += 1
#             except:
#                 pass
#         a = np.array(a)/n
#         a = a.astype(np.float32)
#         return a
#         #query
#         df['word2vec_query_mean'] = df.apply(lambda x: model_word.getvec(x[1].split(' ')).mean(), axis=1)
#         df['word2vec_query_max'] = df.apply(lambda x: model_word.getvec(x[1].split(' ')).max(), axis=1)
#         #title
#         df['word2vec_title_mean'] = df.apply(lambda x: model_word.getvec(x[3].split(' ')).mean(), axis=1)
#         df['word2vec_title_max'] = df.apply(lambda x: model_word.getvec(x[3].split(' ')).max(), axis=1)
#         #query+title
#         df['word2vec_query_title_mean'] = df.apply(lambda x: model_word.getvec(x[1].split(' ') + x[3].split(' ')).mean(), axis=1)
#         df['word2vec_query_title_max'] = df.apply(lambda x: model_word.getvec(x[1].split(' ') + x[3].split(' ')).max(), axis=1)

#     time_elapsed = time.time() - since#######################################
#     print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
#     ite += 1
    
    
# #    #使用doc2vec求句向量的均值和最大值
# #    model_doc = Doc2Vec.load('../1.model')
# #    #query
# #    df['docvec_query_mean'] = df.apply(lambda x: model_doc.infer_vector(x[1].split(' ')).mean(), axis=1)
# #    df['docvec_query_max'] = df.apply(lambda x: model_doc.infer_vector(x[1].split(' ')).max(), axis=1)
# #    #title
# #    df['docvec_title_mean'] = df.apply(lambda x: model_doc.infer_vector(x[3].split(' ')).mean(), axis=1)
# #    df['docvec_title_max'] = df.apply(lambda x: model_doc.infer_vector(x[3].split(' ')).max(), axis=1)
# #    #query+title
# #    df['docvec_query_title_mean'] = df.apply(lambda x: model_doc.infer_vector(x[1].split(' ') + x[3].split(' ')).mean(), axis=1)
# #    df['docvec_query_title_max'] = df.apply(lambda x: model_doc.infer_vector(x[1].split(' ') + x[3].split(' ')).max(), axis=1)


# #    #Doc2vec计算两个list的相似度
# #    def doc2vec_similar(x):
# #        a = x[1].split(' ')
# #        a = [i for i in a if i in model_doc]
# #        b = x[3].split(' ')
# #        b = [i for i in b if i in model_doc]  
# #        return model_doc.n_similarity(a, b)
# #    df['doc_list_similarity'] = df.apply(lambda x: doc2vec_similar(x), axis=1)
    

# #    #doc2vec和word2vec分别得到相似度乘积
# #    df['doc_word_list_similarity_product'] = df['word_list_similarity'] * df['doc_list_similarity']
    

#     #jaro_winkler和jaro的均值、乘积、差值
#     df['jaroWinkler_jaro_mean'] = (df['jaro'] + df['jaro_winkler'])/2
#     df['jaroWinkler_jaro_product'] = df['jaro'] * df['jaro_winkler']
#     df['jaroWinkler_jaro_subtraction'] = (df['jaro'] - df['jaro_winkler']).abs()
    
#     time_elapsed = time.time() - since#######################################
#     print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
#     ite += 1
      
    
#     #Pehrson和cosine_similarity的均值、乘积
#     df['cosine_similarity_Pehrson_mean'] = (df['Pehrson'] + df['cosine_similarity'])/2
#     df['cosine_similarity_Pehrson_product'] = df['Pehrson'] * df['cosine_similarity']
#     df['cosine_similarity_Pehrson_subtraction'] = (df['Pehrson'] * df['cosine_similarity']).abs()
    
#     time_elapsed = time.time() - since#######################################
#     print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
#     ite += 1
    
    
# #    #TF系数
# #    #max
#     def tf(x):
#         x = x.split(' ')
#         x = pd.Series(x).value_counts().values
#         x = x/x.sum()
#         return x
# #    df['query_tf_max'] = df.apply(lambda x: tf(x[1]).max(), axis=1)
#     df['title_tf_max'] = df.apply(lambda x: tf(x[3]).max(), axis=1)
#     df['query_title_tf_max'] = df.apply(lambda x: tf(x[1] + ' ' + x[3]).max(), axis=1)
#     #mean
# #    df['query_tf_mean'] = df.apply(lambda x: tf(x[1]).mean(), axis=1)
#     df['title_tf_mean'] = df.apply(lambda x: tf(x[3]).mean(), axis=1)
#     df['query_title_tf_mean'] = df.apply(lambda x: tf(x[1] + ' ' + x[3]).mean(), axis=1)
    
#     time_elapsed = time.time() - since#######################################
#     print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
#     ite += 1
    
#     #idf系数
#     #max
#     def idf(x):
#         x = x.split(' ')
#         l = len(x)
#         x = pd.Series(x).value_counts().values
#         x = np.log10(l/x)
#         return x
# #    df['query_idf_max'] = df.apply(lambda x: idf(x[1]).max(), axis=1)
#     df['title_idf_max'] = df.apply(lambda x: idf(x[3]).max(), axis=1)
#     df['query_title_idf_max'] = df.apply(lambda x: idf(x[1] + ' ' + x[3]).max(), axis=1)
#     #mean
# #    df['query_idf_mean'] = df.apply(lambda x: idf(x[1]).mean(), axis=1)
#     df['title_idf_mean'] = df.apply(lambda x: idf(x[3]).mean(), axis=1)
#     df['query_title_idf_mean'] = df.apply(lambda x: idf(x[1] + ' ' + x[3]).mean(), axis=1)
         
    
#     time_elapsed = time.time() - since#######################################
#     print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
#     ite += 1
      
#     #TF*IDF#####################################(40)
#     def tfidf(x):
#         t = tf(x)
#         i = idf(x)
#         ti = t*i
#         return ti
#     #max
# #    df['query_tfidf_max'] = df.apply(lambda x: tfidf(x[1]).max(), axis=1)
#     df['title_tfidf_max'] = df.apply(lambda x: tfidf(x[3]).max(), axis=1)
#     df['query_title_tfidf_max'] = df.apply(lambda x: tfidf(x[1] + ' ' + x[3]).max(), axis=1)
#     #mean
# #    df['query_tfidf_mean'] = df.apply(lambda x: tfidf(x[1]).mean(), axis=1)
#     df['title_tfidf_mean'] = df.apply(lambda x: tfidf(x[3]).mean(), axis=1)
#     df['query_title_tfidf_mean'] = df.apply(lambda x: tfidf(x[1] + ' ' + x[3]).mean(), axis=1)
    
#     time_elapsed = time.time() - since#######################################
#     print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
#     ite += 1
           

    
#     #每句的tfidf最大值，平均值
#     t = TfidfVectorizer(ngram_range=(1, 2), analyzer='char',binary=True)
#     title = t.fit_transform(df[3]).toarray()
#     df['title_tfidfvec_mean'] = title.mean(axis=1)
#     df['title_tfidfvec_max'] = title.max(axis=1)
#     del title
#     q_t = t.fit_transform(df[1]+' '+df[3]).toarray()
#     df['query_title_tfidfvec_mean'] = q_t.mean(axis=1)
#     df['query_title_tfidfvec_max'] = q_t.max(axis=1)
#     del q_t
    
#     time_elapsed = time.time() - since#######################################
#     print('{} {} complete in {:.0f}m {:.0f}s'.format(ite, df.columns[-1], time_elapsed // 60, time_elapsed % 60))
#     ite += 1
    
    
###############################################################################    
#    skf = StratifiedKFold(n_splits=5, random_state=2019, shuffle=True)
#    #单维度转化率特征
#    single_fea_set = ['query', 'title']
#    df = get_single_dimension_rate_train_feature(df, single_fea_set)
#    
#    #交叉转化率特征
#    jiaoch_fea_set = ['query', 'title']
#    df = get_jiaocha_dimension_rate_train_feature(df, jiaoch_fea_set)
###############################################################################
    
    

    # try:
    #     df.drop([0, 1, 2, 3, 4], axis=1, inplace=True)
    # except:
    #     df.drop([0, 1, 2, 3], axis=1, inplace=True)
    

    return df

time: 58.3 ms


In [9]:
x = 1
for df in train:
    if x == 1:
        df_train = df.copy()
    if x == 10:
        df = pd.concat((df, df_train))
        break
    print(x)
    x += 1
del df_train
gc.collect()

In [10]:
df['index'] = df.index

y_train = df['label']
y_train.to_csv('feature_label_First1ELast1E_test.txt', index=False)

# #单维度转化率
# single_fea_set = ['query', 'title']
# #训练集
# df = feature.get_single_dimension_rate_train_feature(df, single_fea_set)
# #测试集
# test = feature.get_single_dimension_rate_test_feature(df, test, single_fea_set)

# #交叉维度转化率
# jiaoch_fea_set = ['query', 'title']
# #训练集
# df = feature.get_jiaocha_dimension_rate_train_feature(df, jiaoch_fea_set)
# #测试集
# test = feature.get_jiaocha_dimension_rate_test_feature(df, test, jiaoch_fea_set)

del y_train
gc.collect()

In [11]:
#特征工程（统计特征，距离特征，相似度特征）
df.index = df['index']
df.drop(['index', 'label'], axis=1, inplace=True)

df = pd.concat((df, test))
df.fillna(-1, inplace=True)

df = df.rename(columns = {'query_id':0, 'query':1, 'title_id':2, 'title':3})

In [29]:
if __name__ == '__main__':
    p = multiprocessing.Pool(10)
    process_list = []
    for i in range(10):  
        # p = multiprocessing.Process(target=getfeature,args=(df.iloc[i*22000000:(i+1)*22000000],))
        result = p.apply_async(getfeature,args=(df.iloc[i*2200:(i+1)*2200],))#实例化进程对象
        process_list.append(result)

    p.close()
    p.join()
        



In [32]:
process_list[1].get()

In [34]:
df = pd.concat([i.get() for i in process_list])

In [36]:
df

In [ ]:
df.to_csv('feature_First1ELast1E_test.txt', index=False)